<a href="https://colab.research.google.com/github/bearbearyu1223/llm-fine-tuning-playground/blob/main/eval_fine_tune_whisper_common_voice_zh_tw_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction


## Prepare the Dev Enviroment

First of all, secure a decent compute instance with GPU for training. The script below can be used to veriy if GPU is assigned and also what is its specifications.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jan 28 03:04:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install huggingface_hub==0.19.4
!pip install -q -U accelerate git+https://github.com/huggingface/peft.git
!pip install transformers==4.36.0
!pip install datasets==2.15.0 Tokenizers==0.15.0
!pip install -q bitsandbytes
!pip install openai
!pip install py7zr
!pip install evaluate jiwer
!pip install rouge-score
!pip install librosa
!pip install -U gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Offline Evaluation

### Get the Dataset from HuggingFace Hub

We use the low-resource language zh-TW from [mozilla-foundation/common_voice_16_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_16_0/viewer/yue)  for the fine-tuning task.

In [5]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from datasets import load_dataset, DatasetDict

# load the dataset from the huggingface hub
common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_16_0", "zh-TW", split="train[:50%]")
common_voice["eval"] = load_dataset("mozilla-foundation/common_voice_16_0", "zh-TW", split="validation[:20%]")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_16_0", "zh-TW", split="test[:50%]")

print(common_voice)

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 7084it [00:00, 133181.16it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 15738it [00:00, 157367.39it/s]
Reading metadata...: 31631it [00:00, 158279.97it/s]
Reading metadata...: 47504it [00:00, 146329.62it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4760it [00:00, 127537.75it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 3542
    })
    eval: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 993
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 2484
    })
})


### Load the Adapter Model Checkpoint and Get the Config from the Adapter Model

In [7]:
import torch
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel
from transformers import AutoModelForSpeechSeq2Seq

PEFT_MODEL = "bearbearyu1223/whisper-large-peft-lora-zh_TW-v0.2"

config = PeftConfig.from_pretrained(PEFT_MODEL)

peft_base_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    return_dict=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

adapter_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

In [8]:
print(peft_base_model)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear8bitLt(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear8bitLt(in_featur

In [9]:
peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

In [10]:
print(peft_model)

PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1280)
          (layers): ModuleList(
            (0-31): 32 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=False)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=32, bias=False)
                  )
               

### Process the Dataset for Offline Evaluation

In [11]:
task = "transcribe"
language = "Chinese"

In [12]:
# remove the columns that are not needed for training: we will only use the 'audio' and 'sentence' columns for training
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]
)
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3542
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 993
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2484
    })
})


In [13]:
from datasets import Audio

# signals to datasets to resample audio samples using 16000Hz on the fly the first time that they are loaded
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
common_voice["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/ece0dc224af5a2af5924b6eec9653ab444a408940dfbb740eda0a53087083138/zh-TW_train_0/common_voice_zh-TW_30575781.mp3',
  'array': array([-5.68434189e-12,  5.45696821e-12,  1.09139364e-11, ...,
         -3.43458305e-06, -3.18003003e-06, -4.74580429e-06]),
  'sampling_rate': 16000},
 'sentence': '某個手勢代表覆議'}

In [14]:
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor
model_name_or_path = config.base_model_name_or_path
# get the feature extractor from the model checkpoints
feature_extractor = AutoFeatureExtractor.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# get the tokentizer from the model checkpoints
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path, language=language, task=task)
# get the processor from the model checkpoints
processor = AutoProcessor.from_pretrained(pretrained_model_name_or_path=model_name_or_path, language=language, task=task)
# get the decoer prompt ids from the model checkpoints
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
def prepare_dataset(batch):
  # load and resample audio data and resample in 16kHz
  audio = batch["audio"]

  # compute log-Mel input features from input audio array
  batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

  # encode target text to label ids
  batch["labels"] = tokenizer(batch["sentence"]).input_ids
  return batch

In [16]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/3542 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/993 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2484 [00:00<?, ? examples/s]

In [17]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        # pad the input to max length
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        label_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = label_batch["input_ids"].masked_fill(label_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step, cut bos token as it is appended later
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [18]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Test Model Inference via WebApp

In [19]:
import gradio as gr
from transformers import AutomaticSpeechRecognitionPipeline, pipeline

In [20]:
peft_pipeline = AutomaticSpeechRecognitionPipeline(model=peft_model, tokenizer=tokenizer, feature_extractor=feature_extractor)

The model 'PeftModel' is not supported for . Supported models are ['Pop2PianoForConditionalGeneration', 'SeamlessM4TForSpeechToText', 'SeamlessM4Tv2ForSpeechToText', 'SpeechEncoderDecoderModel', 'Speech2TextForConditionalGeneration', 'SpeechT5ForSpeechToText', 'WhisperForConditionalGeneration', 'Data2VecAudioForCTC', 'HubertForCTC', 'MCTCTForCTC', 'SEWForCTC', 'SEWDForCTC', 'UniSpeechForCTC', 'UniSpeechSatForCTC', 'Wav2Vec2ForCTC', 'Wav2Vec2ConformerForCTC', 'WavLMForCTC'].


In [21]:
base_pipeline = pipeline(model=config.base_model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
def transcribe_base(audio, pipeline=base_pipeline):
    with torch.cuda.amp.autocast():
        text = pipeline(audio)["text"]
    return text

In [23]:
def transcribe_peft(audio, pipeline=peft_pipeline):
    with torch.cuda.amp.autocast():
        text = peft_pipeline(audio)["text"]
    return text

In [24]:
iface_base = gr.Interface(
    fn=transcribe_base,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs="text",
    title="Whisper Large V2",
    description="WebApp Demo for speech recognition using `openai/whisper-large-v2`.",
)

iface_base.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7451c61b3d33b28fce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [25]:
iface_peft = gr.Interface(
    fn=transcribe_peft,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs="text",
    title="Whisper Large V2 PEFT-LoRA-INT8 zh_TW",
    description="WebApp Demo for speech recognition using `bearbearyu1223/whisper-large-peft-lora-zh_TW-v0.2`.",
)

iface_peft.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0442e89b6cba2e95cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### Evaluate the WER on Testset

In [26]:
import evaluate

metric = evaluate.load("wer")

In [27]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)

peft_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                peft_model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=225,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()
wer = metric.compute()
print(f"WER: {wer:.2f}")
# final WER: 0.34

 69%|██████▉   | 215/311 [19:15<08:33,  5.35s/it]Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1561, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyi

WER: 0.33
